In [0]:
%sql
CREATE OR REPLACE TABLE training.ashish.kpi_sales_per_customer AS
SELECT 
    c.CustomerID,
    CONCAT(LEFT(c.FirstName,2),'*** ',LEFT(c.LastName,2),'***') AS CustomerName,
    CONCAT(LEFT(c.AccountNumber,2),'***',RIGHT(c.AccountNumber,2)) AS MaskedAccountNumber,
    CASE 
        WHEN current_user() = 'ashish.sakhare@inferenz.ai' OR is_account_group_member('DataEngineer')
        THEN c.EmailAddress
        ELSE CONCAT(LEFT(c.EmailAddress,2),'*****@*****',RIGHT(c.EmailAddress,2))
    END AS MaskedEmail,
    SUM(s.LineTotal) AS TotalSales,
    SUM(s.OrderQty) AS TotalQuantity,
    COUNT(DISTINCT s.sales_order_id) AS TotalOrders
FROM training.ashish.Sales_fact_table s
JOIN training.ashish.dim_customer c
    ON s.customer_key = c.CustomerID
GROUP BY 
    c.CustomerID, c.FirstName, c.LastName, c.AccountNumber, c.EmailAddress;


In [0]:
%sql
CREATE OR REPLACE TABLE training.ashish.kpi_sales_per_product AS
SELECT 
    p.ProductID,
    p.Product_Name,
    p.ProductModel_Name,
    p.Subcategory_Name,
    p.ProductCategory_Name,
    SUM(s.LineTotal) AS TotalSales,
    SUM(s.OrderQty) AS TotalQuantity,
    COUNT(DISTINCT s.sales_order_id) AS TotalOrders
FROM training.ashish.Sales_fact_table s
JOIN training.ashish.dim_product p
    ON s.product_key = p.ProductID
GROUP BY 
    p.ProductID, p.Product_Name, p.ProductModel_Name, p.Subcategory_Name, p.ProductCategory_Name;


In [0]:
%sql
select * from training.ashish.kpi_sales_per_product

In [0]:
%sql
CREATE OR REPLACE TABLE training.ashish.kpi_sales_by_date AS
SELECT 
    d.Date AS DateKey,
    d.Year,
    d.Month,
    SUM(s.LineTotal) AS TotalSales,
    SUM(s.OrderQty) AS TotalQuantity,
    COUNT(DISTINCT s.order_id) AS TotalOrders
FROM training.ashish.Sales_fact_table s
JOIN training.ashish.dim_date d
    ON s.date_key = d.Date
GROUP BY 
    d.Date, d.Year, d.Month,  d.DayOfWeek;


In [0]:
%sql
select * from training.ashish.kpi_sales_by_date

In [0]:
%sql
CREATE OR REPLACE TABLE training.ashish.kpi_avg_sales_per_customer AS
SELECT 
    s.customer_key,
    ROUND(AVG(s.LineTotal),2) AS AvgSalesPerOrder,
    ROUND(AVG(s.OrderQty),2) AS AvgQuantityPerOrder
FROM training.ashish.Sales_fact_table s
GROUP BY s.customer_key;


In [0]:
%sql
select * from training.ashish.kpi_avg_sales_per_customer

In [0]:
%sql
CREATE OR REPLACE TABLE training.ashish.kpi_top_customers AS
SELECT 
    c.CustomerID,
    CONCAT(LEFT(c.FirstName,2),'*** ',LEFT(c.LastName,2),'***') AS CustomerName,
    SUM(s.LineTotal) AS TotalSales
FROM training.ashish.Sales_fact_table s
JOIN training.ashish.dim_customer c
    ON s.customer_key = c.CustomerID
GROUP BY c.CustomerID, c.FirstName, c.LastName
ORDER BY TotalSales DESC
LIMIT 10;


In [0]:
%sql
select * from training.ashish.kpi_top_customers